# 死锁
所谓死锁： 是指两个或两个以上的进程或线程在执行过程中，因争夺资源而造成的一种互相等待的现象，若无外力作用，它们都将无法推进下去。

此时称系统处于死锁状态或系统产生了死锁，这些永远在互相等待的进程称为死锁进程，如下就是死锁

In [3]:
%%writefile test_thread.py
from threading import Thread,Lock
import time

mutexA=Lock()
mutexB=Lock()

class MyThread(Thread):
    def run(self) -> None:
        self.f1()
        self.f2()
    
    def f1(self) -> None:
        mutexA.acquire()
        print(f"{self.name} 拿到了A锁")
        
        mutexB.acquire()
        print(f"{self.name} 拿到了B锁")
        
        mutexB.release()
        print(f"{self.name} 释放了B锁")
        
        mutexA.release()
        print(f"{self.name} 释放了A锁")

    def f2(self) -> None:
        mutexB.acquire()
        print(f"{self.name} 拿到了B锁")
        time.sleep(0.1)
        
        mutexA.acquire()
        print(f"{self.name} 拿到了A锁")
        
        mutexA.release()
        print(f"{self.name} 释放了A锁")
        
        mutexB.release()
        print(f"{self.name} 释放了B锁")
        
if __name__ == "__main__":
    for _ in range(10):
        m=MyThread()
        m.start()
    

Overwriting test_thread.py


# 使用递归锁解决死锁

解决方法，递归锁，在Python中为了支持在同一线程中多次请求同一资源，python提供了可重入锁RLock。

这个RLock内部维护着一个Lock和一个counter变量，counter记录了acquire的次数，从而使得资源可以被多次require。直到一个线程所有的acquire都被release，其他的线程才能获得资源。上面的例子如果使用RLock代替Lock，则不会发生死锁：

In [4]:


%%writefile test_thread.py
from threading import Thread,RLock
import time

mutexA=mutexB=RLock() #一个线程拿到锁，counter加1,该线程内又碰到加锁的情况，则counter继续加1，这期间所有其他线程都只能等待，等待该线程释放所有锁，即counter递减到0为止

class MyThread(Thread):
    def run(self) -> None:
        self.f1()
        self.f2()
    
    def f1(self) -> None:
        mutexA.acquire()
        print(f"{self.name} 拿到了A锁")
        
        mutexB.acquire()
        print(f"{self.name} 拿到了B锁")
        
        mutexB.release()
        print(f"{self.name} 释放了B锁")
        
        mutexA.release()
        print(f"{self.name} 释放了A锁")

    def f2(self) -> None:
        mutexB.acquire()
        print(f"{self.name} 拿到了B锁")
        time.sleep(3)
        
        mutexA.acquire()
        print(f"{self.name} 拿到了A锁")
        
        mutexA.release()
        print(f"{self.name} 释放了A锁")
        
        mutexB.release()
        print(f"{self.name} 释放了B锁")
        
if __name__ == "__main__":
    for _ in range(10):
        m=MyThread()
        m.start()
    

Overwriting test_thread.py
